In [23]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import unittest
import time
import pandas as pd
import numpy as np
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import json
from lxml import etree


Parametros y listas

In [24]:
lista_anuncios = []

In [25]:
for x in range(1,30):
    driver = webdriver.Safari()
    driver.get("https://www.inmuebles24.com/locales-comerciales-en-renta-en-ciudad-de-mexico-de-1-a-80-metros-cuadrados-cubiertos-pagina-"+str(x)+".html")
    
    contenedor_anuncios = driver.find_element(By.CLASS_NAME, "postings-container")
    contenedor_anuncios_html = contenedor_anuncios.get_attribute("innerHTML")
    soup = BeautifulSoup(contenedor_anuncios_html, "html.parser")
    anuncios = soup.find_all("div")

    for anuncio in anuncios:
        try:
            link = anuncio.find("a")
            print("❤️ Link ->", link.get("href"))
            href = link.get("href")
            lista_anuncios.append("www.inmuebles24.com" + href)
        except:
            pass
    driver.close()

❤️ Link -> /propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html
❤️ Link -> /propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html
❤️ Link -> /propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html
❤️ Link -> /propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html
❤️ Link -> /propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html
❤️ Link -> /propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html
❤️ Link -> /propiedades/clasificado/alcllcin-renta-local-comercial-de-38-m-sup2--en-san-angel-91413236.html
❤️ Link -> /propiedades/clasificado/alcllcin-renta-local-comercial-de-38-m-sup2--en-san-angel-91413236.html
❤️ Link -> /propiedades/clasificado/alcllcin-renta-local-comercial-de-38-m-sup2--en-san-angel-91413236.html
❤️ Link -> /propiedades/clasificado/alcllcin-renta-local-comercial-de-38-m-sup2--en-san-angel-91413236.html
❤️ Link -> /propiedades/clasificado/alcllcin-renta-local-c

In [26]:
lista = lista_anuncios

https://www.inmuebles24.com/locales-comerciales-en-renta-de-1-a-80-metros-cuadrados-cubiertos.html



In [27]:
#eliminar duplicados de la lista
lista = list(dict.fromkeys(lista))
lista

['www.inmuebles24.com/propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-renta-local-comercial-de-38-m-sup2--en-san-angel-91413236.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-renta-local-comercial-de-56-m-sup2--en-la-condesa-90874968.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-excelente-local-en-renta-4-90813187.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-local-en-renta-en-adolfo-ruiz-cortinez-coyoacan-66390530.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-renta-local-comercial-de-66-m-sup2--en-san-angel-91321186.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-local-comercial-en-renta-de-inigualable-ubicacion-59061960.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-renta-barberia-en-av-paseo-de-las-palmas-63996324.html',
 'www.inmuebles24.com/propiedades/clasificado/alcllcin-renta-local-comercial-wtc-piso-2-9112

In [28]:
lista_html = []
for x in lista:
    driver = webdriver.Safari()
    driver.get("https://" + x)
    
    # Esperar hasta que la página esté completamente cargada (puedes ajustar el tiempo máximo según tus necesidades)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    lista_html.append(driver.page_source)
    driver.quit()

In [35]:
lista_html[0]

'<html lang="es_mx"><head>\n\t<title>Local en Renta Cuajimalpa, Ciudad de México - Inmuebles24</title>\n<meta name="keywords" content="Local en renta Cuajimalpa, Ciudad de México Inmuebles24,Local Comercial,Renta,Ciudad de México,Cuajimalpa de Morelos,El Yaqui,Local en Renta Cuajimalpa Terreno 80,Construidos 80,Baño 1,Estacionamientos 2,Antigüedad 10 años inmuebles24 inmuebles">\n<meta name="description" content="Magnifico local en renta, ideal para franquicias, dentro de plaza comercial, cuenta con 80 metros, dentro de la plaza existen marcas de ree...El Yaqui">\n<link rel="canonical" href="https://www.inmuebles24.com/propiedades/clasificado/alcllcin-local-en-renta-cuajimalpa-65607000.html">\n\n\t<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0 shrink-to-fit=no">\n<meta name="format-detection" content="telephone=no">\n<meta name="apple-mobile-web-app-capable" content="yes">\n<meta http-equiv="X-UA-Compatible" content="ie=e

In [36]:
def scrap():
    # de lista_html extraer el div class price-value , section-location-property
# y el div class description-text

    lista_precio = []
    lista_ubicacion = []
    lista_coordenadas = []
    lista_tamanio = []

    for x in lista_html:
        soup = BeautifulSoup(x, "html.parser")
        precio = soup.find("div", class_="price-value")
        tamanio = soup.find("div", class_="article-container-property")
        ubicacion = soup.find("div", class_="section-location-property")

        # Verificar que todos los elementos necesarios estén presentes
        if all(elem is not None for elem in [precio, tamanio, ubicacion]):
            precio_text = re.sub("[^0-9]", "", precio.text) if precio else None
            tamanio_text = re.sub("[^0-9]", "", tamanio.find("h2").text) if tamanio else None
            ubicacion_text = ubicacion.find("h4").text if ubicacion else None

            root = etree.HTML(str(soup))
            coordenadas = root.xpath('//*[@id="static-map"]')
            coordenadas = coordenadas[0].get("src") if coordenadas else None

            coincidencias = re.findall(r"[-+]?\d*\.\d+|\d+", coordenadas) if coordenadas else None

            # Verificar que todas las listas tengan información antes de agregar a las listas
            if all(x is not None for x in [precio_text, tamanio_text, ubicacion_text, coincidencias]):
                lista_precio.append(precio_text)
                lista_tamanio.append(tamanio_text)
                lista_ubicacion.append(ubicacion_text)
                lista_coordenadas.append(coincidencias)

    diccionario = {"precio": lista_precio, "ubicacion": lista_ubicacion, "coordenadas": lista_coordenadas, "tamanio": lista_tamanio}
    df = pd.DataFrame(diccionario)

    return df

In [37]:
a = scrap()

In [38]:
#crear dataset a partir de otro 
df = pd.DataFrame(a)

In [39]:
#quiero mostrar el largo completo de la tabla 
pd.set_option('display.max_rows', None)
# Seleccionar solo los primeros dos elementos de la columna 'coordenadas'
coordenadas_subset = df['coordenadas'].apply(lambda x: x[:2] if x else None)

# Agregar la nueva columna al DataFrame
df['coordenadas_subset'] = coordenadas_subset

# Mostrar el DataFrame con la nueva columna
(df)

,precio,ubicacion,coordenadas,tamanio,coordenadas_subset
0,29900,"Jose Maria Castorena 232, El Yaqui, Cuajimal...","[19.361768000000000, -99.288325600000000, 16, ...",802,"[19.361768000000000, -99.288325600000000]"
1,25000,"Avenida de la Paz 40, San Ángel, Alvaro Obregón","[19.347110300000000, -99.189234600000000, 16, ...",3820,"[19.347110300000000, -99.189234600000000]"
2,62000,"Avenida Baja California 295, Condesa, Cuauht...","[19.406041400000000, -99.172278200000000, 16, ...",56,"[19.406041400000000, -99.172278200000000]"
3,8000,"BOLAÑOS, Felipe Pescador, Cuauhtémoc","[19.454100000000000, -99.125200000000006, 16, ...",30,"[19.454100000000000, -99.125200000000006]"
4,16000,"Huitzilopochtli Lte 14 Mz 19, Adolfo Ruiz Co...","[19.320184600000001, -99.150290499999996, 16, ...",35,"[19.320184600000001, -99.150290499999996]"
5,50000,"AV. DE LA PAZ 40, San Ángel, Alvaro Obregón","[19.347110300000000, -99.189234600000000, 16, ...",66,"[19.347110300000000, -99.189234600000000]"
6,9000,"Lucerna 65, Juárez, Cuauhtémoc","[19.430266400000000, -99.156914700000000, 16, ...",8,"[19.430266400000000, -99.156914700000000]"
7,50000,"Avenida de las Palmas 1065, Lomas de Chapult...","[19.426298600000000, -99.221970100000000, 16, ...",291,"[19.426298600000000, -99.221970100000000]"
8,18000,"Montecito 38, Nápoles, Benito Juárez","[19.394554280612800, -99.173986511639400, 16, ...",40,"[19.394554280612800, -99.173986511639400]"
9,55000,"AV. DE LA PAZ 40, San Ángel, Alvaro Obregón","[19.347110300000000, -99.189234600000000, 16, ...",64,"[19.347110300000000, -99.189234600000000]"


In [40]:
#eliminar la columna coordenadas y coordenadas_subset
df.drop(['coordenadas'], axis=1, inplace=True)
df.drop(['coordenas'], axis=1, inplace=True)

KeyError: "['coordenas'] not found in axis"

In [42]:
#separar la columna coordenas_subset en dos columnas latitud y longitud
df['latitud'] = df['coordenadas_subset'].apply(lambda x: x[0] if x else None)
df['longitud'] = df['coordenadas_subset'].apply(lambda x: x[1] if x else None)

#eliminar la columna coordenas
df.drop(['coordenadas_subset'], axis=1, inplace=True)


In [43]:
df.head(3)

,precio,ubicacion,tamanio,latitud,longitud
0,29900,"Jose Maria Castorena 232, El Yaqui, Cuajimal...",802,19.361768000000000,-99.288325600000000
1,25000,"Avenida de la Paz 40, San Ángel, Alvaro Obregón",3820,19.347110300000000,-99.189234600000000
2,62000,"Avenida Baja California 295, Condesa, Cuauht...",56,19.406041400000000,-99.172278200000000


In [44]:
#exportar a csv
df.to_csv('inmuebles24_2.csv', index=False)

In [47]:
#necesito simplificar las coordenadas para poder graficarlas , entonces solo seleccionare los dos primeros numeros de la coordenada y dos numeros decimales de la latitud y longitud
df['latitud'] = df['latitud'].apply(lambda x: x[:6] if x else None)
df['longitud'] = df['longitud'].apply(lambda x: x[:6] if x else None)




In [50]:
print(df.head(3))

  precio                                          ubicacion tamanio latitud  \
0  29900   Jose Maria Castorena 232,  El Yaqui, Cuajimal...     802  19.361   
1  25000   Avenida de la Paz 40,  San Ángel, Alvaro Obregón    3820  19.347   
2  62000   Avenida Baja California 295,  Condesa, Cuauht...      56  19.406   

  longitud  
0   -99.28  
1   -99.18  
2   -99.17  


In [49]:
#exportar a csv
df.to_csv('inmuebles24_coordenas_simplificadas.csv', index=False)